# Histogram of Oriented Gradients
We explore the method of 'histogram of oriented gradients' (HOG) as described in Navneet Dalal and Bill Triggs paper entitled <u>Histograms of Oriented Gradients for Human Detection</u>. We will be using the INRIA Dataset, specifically those images found in 160x96 folder to test our implementation. The code to process the data we will be written in `Python`. In addition we will show how to use the HOG found in `OpenCV`.

## Peope Detection
To be able to detect people in an image we follow the approach outline in <u>Histograms of Oriented Gradients for Human Detection</u>. The idea presented in the paper is to capture the objects appearance and shape by characterizing it using local intensity gradients and edge directions. The image is densely divided into small spacial regions called cells. Each cell is then grouped into a block consisting of 4 cell arrange in a 2x2 configuration. The 160x96 pixel image divided into 19x11 blocks such that each block has a 50% overlap with another block. Within each of the blocks each cell is used to generate a 1-D histogram of gradient directions/edge directions and later all cell data is combined to give a complete histogram of oriented gradients descriptor of the window, resulting in a 19x11x(4*9) = 4930-D vector for a histogram utilizing 9 bins. The entire process is represented by the image below. 

<img src="HOG_Steps.png" alt="" style="width:auto; height:auto;">

(<i>In our case we will not be normalizing gamma nor we will be normalizing the colours, we will also be converting the images into grayscale before processing them further.</i>)

## Code
### Required Libraries

In [1]:
# Libraries
# Standard Libraries
import os
from PIL import Image

# Third Party Libraries
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
from sklearn import svm
from sklearn.model_selection import KFold

%matplotlib inline

### Compute Gradients
The gradient is computed as a point discrete derivative mask. In their paper Dalal and Triggs describe various methods to compute the gradient and they discovered that using a 1-D $[−1, 0, 1]$ masks in both horizontal and vertical directions at $\sigma=0$ work best. (The mask that were tested include an uncentred $[−1, 1]$, centred $[−1, 0, 1]$ and cubic corrected $[1, −8, 0, 8, −1]$, in addition to a 3×3 Sobel masks and 2×2 diagonal ones $\left(\begin{array}{rr}
0 & 1 \\ 
-1 & 0 \end{array} \right)$ and $\left(\begin{array}{rr}
-1 & 0 \\ 
0 & 1 \end{array} \right)$).

Below we show how to generate the mask by defining a `grad` functions that takes the image as an argument. 

In [1]:
def grad(image):
    grad_hor, grad_ver = np.zeros_like(image), np.zeros_like(image)

    grad_hor = np.gradient(image, axis=1)
    grad_hor[:, 1:-1] = (2 * grad_hor[:, 1:-1]) % 256

    grad_ver = np.gradient(image, axis=0)
    grad_ver[1:-1, :] = (2 * grad_ver[1:-1, :]) % 256

    return grad_hor, grad_ver

In addition to computing the gradients we also need to compute the magnitudes and orientation. As per the findings in the paper we choose to go with an unsigned orientation thus limiting our angle to be between $0^{\circ}$ and $180^{\circ}$. 

In [3]:
def magnitude_orientation(gx, gy):
    magnitude = np.hypot(gx, gy)
    orientation = (arctan2(gy, gx) * 180 / np.pi) % 180

    return magnitude, orientation

### Weigthed Vote Into Spatial & Orientation Cells
This stage involves dividing the window into adjacent, non-overlapping regions called cells of size $C\times C$ pixel. For simplicity we choose to use rectangular cells, nevertheless, there is no restriction to the shape of a cell and Dalal and Triggs give a description of a radial (log-polar sectors) cell.  Within each cell "each pixel calculates a weighted vote for an edge orientation histogram channel based on the orientation of the gradient element centred on it, and the votes are accumulated into orientation bins over local spatial regions. The orientation bins are evenly spaced over $0^{\circ}$ to $180^{\circ}$. To reduce aliasing, votes are interpolated bilinearly between the neighbouring bin centres in both orientation and position. The vote is a function of the gradient magnitude at the pixel, either the magnitude itself, its square, its square root, or a clipped form of the magnitude representing soft presence/absence of an edge at the pixel".

<b>Example: </b> 
To compute the weigthed votes let $B$ be the number of bins (in our case $B=9$), $m$ the magnitude, and $\theta$ the orientation. We number the bins from $0$ to $B−1$ where each bin has width $w = \frac{180}{B} \big(\frac{180}{9} = 20\big)$. Bin $i$ has boundaries $[wi, w(i + 1))$ (in our case the boundary is $[20i, 20(i+1))$). The vote for a pixel is computed to be:

<h5 align="center">
$
\begin{align}
v_j &= m\frac{w(j+1) − \theta}{w} \text{ to bin number } j \\
v_{j+1} &= m\frac{w(j+2) − \theta}{w} \ \text{ to bin number } (j+1)
\end{align}
$
</h5>

More concretely, if we let $\theta = 77^{\circ}$, $B=9$, $w=20$ then 
<h5 align="center">
$
\begin{align}
v_3 &= m\frac{w_4 − \theta}{20} \text{ to bin number } j = 3 \\
&= m\frac{80-77}{20} = m\frac{3}{20} = 0.15m\\
\text{and}&\\
v_{4} &= m\frac{w_5 - \theta}{20} \ \text{ to bin number } j =4 \\
&= m\frac{90 - 77}{20} = m\frac{17}{20} = 0.85m
\end{align}
$
</h5>
<img src="bilinear_inter2.png" alt="" style="width:auto; height:auto;">

To compute the "weigthed vote into spatial & orientation cells" we first need to build the cell which should take as arguments the orientation, the gradient, and the number of bins, this is done below in the `_build_cell` function. We can then define a `_build_hist` functions whih takes the same arguments as the `_build_cell` function in addition to cell size to build a histogram. We choose to compute the vote using the magnitude simalarly to the example above.

In [4]:
def _build_cell(orientation, gradient, nbins):

    cell_histogram = np.zeros(shape=nbins)
    b_step = 180/nbins

    for orient, grad in zip(orientation, gradient):
        for o, g in zip(orient, grad):
            c_bin = np.int8(np.abs(o//20))
            prop = np.abs(o % b_step) / b_step

            if c_bin == (nbins -1):
                cell_histogram[c_bin] += (1-prop) * g
                cell_histogram[0] += prop * g
            else:
                cell_histogram[c_bin] += (1-prop) * g
                cell_histogram[c_bin+1] += prop * g

    return cell_histogram

def _build_hist(orientation, gradient, nbins, cell_size):

    cell_y, cell_x = cell_size    
    overfill = tuple([s % c for s, c in zip(orientation.shape, cell_size)])

    # Gets the left side of the image, if the image cannot be covered by
    # equal sized cells
    shape = tuple([s_i - over_i 
                   for s_i, over_i in zip(orientation.shape, overfill)])

    histograms = np.zeros((shape[0]//cell_y, shape[1]//cell_x, nbins))
    i = 0
    for y in range(0, shape[0], cell_y):
        j = 0
        for x in range(0, shape[1], cell_x):
            orient = orientation[y: y+cell_y, x: x+cell_x]
            grad = gradient[y: y+cell_y, x:x+cell_x]
            histograms[i, j, :] += _build_cell(orient, grad, nbins=nbins)
            j += 1
        i+= 1

    return histograms

### Contrast Normalize Over Overlapping Spatial Blocks

Gradient strengths vary over a wide range owing to local variations in illumination and foreground-background contrast, as such it is necessary to apply some form of normalization. Normalization is accomplished by grouping cells into overlapping blocks of $2\times2$ cells, so that each block is of size size $2C×2C$ pixels. Two horizontally or vertically consecutive blocks overlap by two cells, that is, the block stride is C pixels. As a consequence, each internal cell is included into four seperate blocks. 

#### Normalization
Dalal and Triggs evaluate four different normalization schemes. If we let $\mathbf{v}$ be the unnormalized descriptor vector,
$||v||_k$ be its k-norm for $k=1, 2$, and $\varepsilon$ be a small constant then the normalizations are described as follows:

<h5 align="center">
$
\begin{align}
L_1 &= \frac{v}{||v||_1 +\varepsilon}\\
L_{1,sqrt} &= \sqrt{\frac{v}{||v||_1 +\varepsilon}}\\
L_2 &= \frac{v}{||v||_2 +\varepsilon}\\
L_{2,clip} &= L_2 \text{ followed by clipping and renormalizing}
\end{align}
$
</h5>

The authors claim that $L_{2,clip}, L_2$ and $L_{1,sqrt}$ all perform equally well, while simple $L_1$ reduces performance by 5%. As such we decide to use $L_2$ as a normalization scheme. 

The `_build_block` function defined bellow describes how to compute this stage using $L_2$ normalization, it takes the same arguments as `_build_hist` in addition to block_size. The function works by sliding over the orientation and gradients to compute the histograms of the cells and grouping them into appropriate size. 

In [2]:
def _build_block(orientation, gradient, nbins, cell_size, block_size):

    histograms = _build_hist(orientation, gradient, nbins, cell_size)
    nbr_y, nbr_x, _ = histograms.shape
    dim = reduce(lambda x, y: x*y, block_size)
    hog_matrix = np.zeros(((nbr_y-1)*(nbr_x-1), dim*nbins))

    i = 0 
    for y in range(0, nbr_y-1):
        j = 0

        for x in range(0, nbr_x-1):
            block = histograms[y: y+block_size[0], x: x+block_size[1]]
            norm_value = np.linalg.norm(block.ravel() + 1e-4, ord=2)
            block = block.ravel()/norm_value
            position = (nbr_x-1)*i + j
            hog_matrix[position] = block
            j += 1
        i+= 1

    return hog_matrix

## HOG Descriptor
Putting it all together we have the following function

In [1]:
def hog(image, nbins=9, cell_size=(8, 8), block_size=(2, 2), flatten=True):
    gx, gy = grad(image)
    mag, orientation = magnitude_orientation(gx, gy)

    hog_matrix = _build_block(orientation=orientation, gradient=mag, 
                              nbins=nbins, cell_size=cell_size, 
                              block_size=block_size)

    if flatten:
        return hog_matrix.ravel()
    else:
        return hog_matrix

### Linear SVM
The article describes using a linear SVM for training. We train the SVM using the sklearn library and we use K-fold cross validation to reduce the change of overfitting. 

### People Detection Using a Sliding Window
Finally, to be able to detect people in an image we use a sliding window approach which we define below.

In [4]:
def sliding_window(image, window_size, step_size):

    for y in range(0, image.shape[0], step_size):
        for x in range(0, image.shape[1], step_size):
            # yield the current window
                img = image[y: y+window_size[1], x: x+window_size[0]]
                if img.shape == (int(window_size[1]), int(window_size[0])):

                    yield (x, y, img)

## Running The Code
To run the code simply load the positive and negative training images, run `hog()` on each image to get the hog descriptor. Once the images have been processed simply call  

To improve detection we can generate an image pyramid and apply a sliding window to each sub-image. We would then combine the overlapped boxes to produce our final detection. This can be done using a mean-shift algorithm to detect multiple modes in the bounding box space by utilizing the (x, y) coordinates of the bounding box as well as the logarithm of the current scale of the image as suggested by Dalal and Triggs. One can also use a non-maximum suppression algorithm to combine the overlapped boxes, we found that this yielded slightly better results  and can be found in the code utilizing `OpenCv`. 

## Using OpenCV
The implimentation in OpenCV is more optimized then that presented above. Furthermore, `OpenCV` has a pretrained people detector, which facilitates the entire process. The code below shows how to use `OpenCV` to perform people detecting.

In [7]:
import cv2
import imutils
from imutils.object_detection import non_max_suppression
import numpy as np

In [8]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

<table style="width:100%">
  <tr>
    <th><img src="People.png" alt="Original"></th>
    <th><img src="people_detect.png" alt="Original"></th> 
  </tr>
 </table>